In [3]:
# Pre configurations

# stable outputs across the runs
import numpy as np
np.random.seed(42)

#folder to the figures
import os
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "classification"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [4]:
#Get MNIST dataset from scikit
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
#check de features
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [8]:
# check matrix

X, y = mnist['data'], mnist['target']
X.shape

#70000 images with 784 characteristics (28x28 pixels)

(70000, 784)

In [9]:
y.shape
#a instance for image

(70000,)

In [ ]:
#check a digit

import matplotlib as mpl
import matplotlib.pyplot as plt

a_digit = X[1]
